In [9]:
from typing import Optional, Dict, Any
from datetime import datetime
from sqlmodel import SQLModel, Field, Column
from sqlalchemy import JSON
from sqlmodel import SQLModel, create_engine, Session
from sqlmodel import select

In [3]:
class Note(SQLModel, table=True):
    __tablename__ = "notes"

    id: Optional[int] = Field(default=None, primary_key=True)

    filepath: str = Field(
        index=True,
        description="Absolute or relative path of the note file"
    )

    note_name: str = Field(
        index=True,
        description="Display name of the note"
    )

    created_time: datetime = Field(
        default_factory=datetime.utcnow,
        description="Note creation time (UTC)"
    )

    modified_time: datetime = Field(
        default_factory=datetime.utcnow,
        description="Last modified time (UTC)"
    )

    other_attr: Dict[str, Any] = Field(
        default_factory=dict,
        sa_column=Column(JSON),
        description="Flexible JSON metadata"
    )

In [5]:
DATABASE_URL = "sqlite:///notes.db"

engine = create_engine(
    DATABASE_URL,
    echo=False,
    connect_args={"check_same_thread": False},  # SQLite + FastAPI
)

def get_session():
    return Session(engine)

In [7]:
SQLModel.metadata.create_all(engine)

In [10]:
class NoteRepository:
    def __init__(self, session: Session):
        self.session = session

    def create(
        self,
        filepath: str,
        note_name: str,
        other_attr: Optional[Dict[str, Any]] = None,
    ):
        note = Note(
            filepath=filepath,
            note_name=note_name,
            other_attr=other_attr or {},
        )
        self.session.add(note)
        self.session.commit()
        self.session.refresh(note)
        return note
    
    def update(
        self,
        note_id: int,
        *,
        filepath: Optional[str] = None,
        note_name: Optional[str] = None,
        other_attr: Optional[Dict[str, Any]] = None,
    ) -> Optional[Note]:
        note = self.session.get(Note, note_id)
        if not note:
            return None

        if filepath is not None:
            note.filepath = filepath

        if note_name is not None:
            note.note_name = note_name

        if other_attr is not None:
            note.other_attr = other_attr

        note.modified_time = datetime.utcnow()

        self.session.add(note)
        self.session.commit()
        self.session.refresh(note)
        return note


    def delete(self, note_id: int) -> bool:
        note = self.session.get(Note, note_id)
        if not note:
            return False

        self.session.delete(note)
        self.session.commit()
        return True

    def get(self, note_id: int) -> Optional[Note]:
        return self.session.get(Note, note_id)
    

    def list_all(self):
        statement = select(Note).order_by(Note.modified_time.desc())
        return self.session.exec(statement).all()